In [ ]:
import chess
import chess.engine
import chess.svg
import chess.pgn
from IPython.display import SVG, display
import time
from engine import MyEngine

import data.download

%load_ext autoreload
%autoreload 2

We will now download move tables to ensure optimal play during the opening and the endamge (up to 5 remaining pieces)

Make sure you are OK with the data before downloading/using it as these files are downloaded from 3rd partys and I don't claim responsibility for the integrity of them

In [ ]:
data.download.download_syzygy_from_url()
data.download.dowload_polyglot_from_url()
data.download.download_stockfish_from_url()

We will create 2 engines one for each player (one is our engine the other is the current version of the stock fish engine)

In [38]:
engine = chess.engine.SimpleEngine.popen_uci("./data/stockfish/stockfish_15_linux_x64_ssse/stockfish_15_x64_ssse")
myengine = MyEngine()

In [35]:

board = chess.Board()
board_list = [board.copy()]
move_list = [chess.Move().null()]

while not board.is_game_over():
    if board.turn == chess.WHITE:
        move = myengine.play(board, 2)
    else:
        # move = engine.play(board, 2)
        move = myengine.play(board, 2)

    board.push(move)
    move_list.append(move)
    board_list.append(board.copy())

engine.quit()

In [49]:
import ipywidgets as widgets
import data.database

move_num = widgets.IntSlider(description='move_num', max=len(board_list)-1)

myengine = MyEngine()


def display_board(move_num):
    board = board_list[move_num]
    move = move_list[move_num]
    display(SVG(chess.svg.board(board, size = 400, lastmove=move)), clear=False)


def engine_score(move_num):
    board = board_list[move_num]
    myscore = myengine.evaluate(board_list[move_num], 2)
    score = engine.analyse(board_list[move_num], limit=chess.engine.Limit(time=2.0))["score"].white().score(mate_score = 10000)
    score = data.database.normalize(score)
    
    print(f"stockfish: {score}")
    print(f"my engine: {myscore}")

    
board_svg = widgets.interactive_output(display_board, {'move_num': move_num})
score = widgets.interactive_output(engine_score, {'move_num': move_num})

widgets.VBox([widgets.VBox([move_num, score]), board_svg])